In [228]:
import pandas as pd
import numpy as np
import math
from tqdm import tqdm
from os.path import join
from utils import get_repo_dir
from config.config import *
from typing import Union

import torch
from torch.utils.data import TensorDataset, DataLoader
from torch import nn
from torch.nn import functional as F

## Some initial variables and data loading

In [216]:
max_num_frames = 30  # truncate if snap-to-throw is > this. units: ds
out_dir = join(get_repo_dir(), 'output')

In [187]:
# load data
week1_tracking = pd.read_csv(join(data_dir, 'week1_norm.csv'))
week1_coverage = pd.read_csv(join(data_dir, 'coverages_week1.csv'), dtype={'coverage': 'category'})

In [188]:
# merge data
week1_data = pd.merge(week1_tracking, week1_coverage, how='right', on=['gameId', 'playId'])
week1_data['coverage_code'] = week1_data.coverage.cat.codes

TODO data aug. this is very slightly complicated by the fact that we have precomputed the vx, vy, ax, ay. will have to ask udit about how this is computed since it seems to be inconsistent with v_theta and v_mag?

for now we don't have orientation relative to qb, so I don't think it makes sense to highlight the qb in the way the bdb winner does for rusher, since it would just be distance from qb and qb-relative speed/accel data right now, which I doubt is _extra_ useful for predicting coverage (?). so for now, we'll do 11x11 of off x def with features:
* relative: x, y, vx, vy, ax, ay
* absolute: vx, vy, ax, ay

## Dataset generation

First, we generate the dataset as numpy arrays.

In [212]:
num_features = 10

grouped = week1_data.groupby(['gameId', 'playId'])
# TODO can change 11s to max number of off and def players in data
dataX = np.empty((len(grouped), max_num_frames, num_features, 11, 11))  # (P, T, F, D, O): play, frame, feature, def, off
dataDims = np.empty((len(grouped), 3)) # (P, 3) contains (t, d, o): num frames, num def, num off on each play
dataY = np.empty(len(grouped))  # (P,)

valid_plays = 0
for play_idx, (game_play, play_df) in tqdm(enumerate(grouped)):
    if 'pass_forward' not in play_df.event.unique():
        continue
    first_frame = play_df.loc[(play_df.nflId == 0) & (play_df.event == 'ball_snap')].frameId.iloc[0]
    play_end_frame = play_df.loc[(play_df.nflId == 0) & (play_df.event == 'pass_forward')].frameId.iloc[0]
    last_frame = min(first_frame + max_num_frames - 1, pass_forward_frame)
    play_df = play_df.loc[play_df.frameId.between(first_frame, last_frame)]
    
    num_def, num_off = 0, 0
    for frame_idx, (frame_id, frame_df) in enumerate(play_df.groupby('frameId')):
        def_ids = frame_df[frame_df.team_pos == 'DEF'].index
        num_def = len(def_ids)
        off_ids = frame_df[frame_df.team_pos == 'OFF'].index
        num_off = len(off_ids)
    
        outer_sub = np.subtract.outer(
            frame_df.loc[off_ids, ['x', 'y', 'v_x', 'v_y', 'a_x', 'a_y']].values,
            frame_df.loc[def_ids, ['x', 'y', 'v_x', 'v_y', 'a_x', 'a_y']].values
        )
        # einsum explanation: the two i's get rid of subtraction across cols
        # k before j reorders def before off since output dims in alph. order
        if num_def > 11 or num_off > 11:
            breakpoint()
        dataX[play_idx, frame_idx, :6, :num_def, :num_off] = np.einsum('kiji->ijk', outer_sub)
        dataX[play_idx, frame_idx, -4:, :num_def, :num_off] = frame_df.loc[def_ids, ['v_x', 'v_y', 'a_x', 'a_y']].values.T[...,None]
    dataY[play_idx] = play_df.coverage_code.iloc[0]
    dataDims[play_idx] = last_frame - first_frame, num_def, num_off
    valid_plays += 1
dataX = dataX[:valid_plays]
dataDims = dataDims[:valid_plays]
dataY = dataY[:valid_plays]
    

1028it [02:00,  8.56it/s]


In [220]:
# uncomment to save
data_save_path = join(out_dir, 'week1_data')
# np.savez(data_save_path, x=dataX, dims=dataDims, y=dataY)
# uncomment to load from save
saved_data = np.load(data_save_path)
dataX, dataDims, dataY = saved_data['x'], saved_data['dims'], saved_data['y']

Next, we make a [TensorDataset](https://pytorch.org/docs/stable/_modules/torch/utils/data/dataset.html#TensorDataset) out of it.

In [226]:
tensorX = torch.Tensor(dataX)
tensorDims = torch.Tensor(dataDims)
tensorY = torch.Tensor(dataY)
dataset = TensorDataset(tensorX, tensorDims, tensorY)
dataloader = DataLoader(dataset)

## Model construction

In [221]:
class DeepCoverInner(nn.Module):
    def __init__(self,
                 input_channels: int,
                 output_dim: int,
                 conv_h: Union[int, Tuple[int, int]]=[128, 96],
                 linear_h: Union[int, Tuple[int, int]]=96, 256)
        """
        :param input_channels: number of input features
        :param output_dim: dimension of output embedding
        :param conv_h: number of conv channels for each conv block.
            int or tuple of 2 ints.
        :param linear_h: number of hidden units for each linear layer.
            int or tuple of 2 ints.    
        """
        if type(conv_h) is int:
            conv_h = (conv_h, conv_h)
        if type(linear_h) is int:
            linear_h = (linear_h, linear_h)
        self.conv_block_1 = nn.Sequential([
            nn.Conv2d(input_channels, conv_h[0], kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(conv_h[0], conv_h[0], kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(conv_h[0], conv_h[0], kernel_size=1)
            nn.ReLU()
        ])
        self.bn1 = nn.BatchNorm1d(conv_h)
        self.conv_block_2 = nn.Sequential([
            nn.Conv1d(conv_h[0], conv_h[1], kernel_size=1),
            nn.ReLU(),
            nn.BatchNorm1d(conv_h),
            nn.Conv1d(conv_h[1], conv_h[1], kernel_size=1),
            nn.ReLU(),
            nn.BatchNorm1d(conv_h),
            nn.Conv1d(conv_h[1], conv_h[1], kernel_size=1)
            nn.ReLU(),
            nn.BatchNorm1d(conv_h),
        ])
        self.conv_h = conv_h
        self.linear_block = nn.Sequential([
            nn.Linear(conv_h[1], linear_h[0]),
            nn.ReLU(),
            nn.BatchNorm1d(linear_h[0]),
            nn.Linear(*linear_h),
            nn.ReLU(),
            nn.LayerNorm(linear_h[1]),
            nn.Linear(linear_h[1], linear_h[2])
        ])
        
        
        
    def forward(self, x, dims):
        # let (F', F") and (..., F*) be conv_h and linear_h args to __init__
        orig_shape = x.shape  # (B, T, F, D, O)
        x = x.view(-1, orig_shape[2:])  # (B*T, F, D, O)
        
        x = self.conv_block_1(x)  # (B*T, F', D, O)
        x = x.view(*orig_shape[:2], *x.shape[1:])  # (B, T, F', D, O)
        # this block of code handles variable number of offensive players
        x_max = torch.stack([
            F.max_pool2d(each[...,:dim[2]], kernel_size=(1, dim[2])).squeeze() for each, dim in zip(x, dims)
        ])  # (B, T, F', D)
        x_avg = torch.stack([
            F.avg_pool2d(each[...,:dim[2]], kernel_size=(1, dim[2])).squeeze() for each, dim in zip(x, dims)
        ])  # (B, T, F', D)
        x = (x_max * 0.3 + x_avg * 0.7).view(-1, *x_max[2:])  # (B*T, F', D)
        x = self.bn1(x)
        
        x = self.conv_block_2(x)
        x = x.view(*orig_shape[:2], *x.shape[1:])  # (B, T, F", D)
        # this block of code handles variable number of defensive players
        x_max = torch.stack([
            F.max_pool1d(each[...,:dim[1]], kernel_size=dim[1]).squeeze() for each, dim in zip(x, dims)
        ])  # (B, T, F")
        x_avg = torch.stack([
            F.avg_pool1d(each[...,:dim[1]], kernel_size=dim[1]).squeeze() for each, dim in zip(x, dims)
        ])  # (B, T, F")
        x = (x_max * 0.3 + x_avg * 0.7).view(-1, *x_max[2:])  # (B*T, F")
        
        x = self.linear_block(x)  # (B*T, F*)
        
        # restore shape
        x = x.view(*orig_shape[:2], -1)  # (B, T, F*)
        return x


class DeepCoverOuter(nn.Module):
    def __init__(self):
        pass
    
    def forward(self, x, dims):
        # x is (B, T, F*)
        pass
    

class DeepCover(nn.Module):
    def __init__(self,
                 inner_args,
                 outer_args):
        self.outer = DeepCoverOuter(**outer_args)
        self.inner = DeepCoverInner(**inner_args)
    
    def forward(self, x, dims):
        x = self.inner(x, dims)
        x = self.outer(x, dims)
        return x

# Testing playground (Junk below here)

In [149]:
a = np.random.randn(20, 5) + 5
b = np.random.randn(20, 5)
# result1 = np.empty((1, 20, 20, 5))
# result2 = np.empty((1, 20, 20, 5))
path = np.einsum_path('ikjk->', np.empty((20,5,20,5)), optimize='optimal')[0]

In [152]:
%%timeit -n100 -r10
subout = np.subtract.outer(a, b)
result1 = np.empty((1, 20, 20, 5))
# path = np.einsum_path('ikjk->', subout, optimize='optimal')[0]
result1[0] = np.einsum('ikjk->', subout)

83.8 µs ± 29.4 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [151]:
%%timeit -n100 -r10
result2 = np.empty((1, 20, 20, 5))
for i in range(a.shape[0]):
    result2[0, i] = a[i] - b

79.1 µs ± 26.5 µs per loop (mean ± std. dev. of 10 runs, 100 loops each)


In [167]:
frame_df = week1_data[(week1_data.gameId == 2018090600) & (week1_data.playId == 75) & (week1_data.frameId == 1)]
# frame_df.values.T[...,None].shape

# test = frame_df[frame_df.team_pos == 'DEF'].index
# len(test)
# frame_df

(32, 14, 1)